In [1]:
!jt -t l

Didn't recognize theme name: l
Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [31]:
import requests, json, pandas as pd, numpy as np

In [3]:
## ajouter publisher_s, journalPublisher_s, anrProjectAcronym_s

hal_req = """
https://api.archives-ouvertes.fr/search/?q=fulltext_t:\
(("applied a CC" AND "BY public copyright") OR ("appliqué une licence" AND CC AND BY))\
&rows=5000&sort=submittedDate_tdate desc&\
fl=halId_s,doiId_s,docType_s,primaryDomain_s,journalIssn_s,submittedDate_s,anrProjectAcronym_s,journalPublisher_s,publisher_s
"""

In [4]:
req_raw = requests.get(hal_req)
req_json = req_raw.json()
print("total of publications", req_json["response"]["numFound"])

total of publications 55


In [5]:
req_data = req_json["response"]["docs"]
df = pd.DataFrame(req_data)
df.sample(1)

,primaryDomain_s,journalPublisher_s,journalIssn_s,halId_s,docType_s,doiId_s,submittedDate_s,anrProjectAcronym_s
36,sdv.bc.bc,NaN,NaN,hal-03454274,UNDEFINED,10.1101/2021.09.24.461707,2021-11-29 10:14:16,NaN


In [6]:
# re order column
#df = df[["halId_s", "doiId_s", "docType_s", "journalIssn_s", "submittedDate_s"]]

In [39]:
## lighten submittedDAte
df["submittedDate_s"] = df["submittedDate_s"].str.slice(0,10)
df.sample(1)

,primaryDomain_s,journalPublisher_s,journalIssn_s,halId_s,docType_s,doiId_s,submittedDate_s,anrProjectAcronym_s,upw_issns,upw_vor_is_oa
2,sdv.mhep.mi,MDPI,2076-0817,hal-03788705,ART,10.3390/pathogens11091045,2022-09-26,NaN,2076-0817,True


In [42]:
## lighten anrProject
df["anrProjectAcronym_s"] = df["anrProjectAcronym_s"].notna()

In [8]:
df["docType_s"].value_counts()

ART          48
UNDEFINED     6
COMM          1
Name: docType_s, dtype: int64

## Enrich w unpaywall

In [9]:
def get_upw(doi):
    # memo https://api.unpaywall.org/v2/10.1101/2021.05.24.445406?email=m@larri.eu
    req = requests.get(f"https://api.unpaywall.org/v2/{doi}?email=m@larri.eu")
    res = req.json()
    
    # get issns
    issns = res.get("journal_issns")
    
    # deduce if vor is oa
    vor_is_oa = False
    if res.get("best_oa_location"):
        best_loc = res["best_oa_location"]
        vor_is_oa = True if best_loc["version"] == "publishedVersion" else False
    
    return [issns, vor_is_oa]

In [10]:
## pour tester
# memo closed doi 10.1017/S135577181900030X
get_upw("10.1101/2021.05.24.445406")

[None, True]

In [11]:
## add upw_issn & upw_vor_is_oa
df[["upw_issns", "upw_vor_is_oa" ]] = df.apply(
    lambda row : get_upw(row["doiId_s"]), axis = 1, result_type = "expand")

In [22]:
df[df["anrProjectAcronym_s"].notnull()]

,primaryDomain_s,journalPublisher_s,journalIssn_s,halId_s,docType_s,doiId_s,submittedDate_s,anrProjectAcronym_s,upw_issns,upw_vor_is_oa
8,sdv.bbm.bc,Elsevier,0955-0674,pasteur-03716706,ART,10.1016/j.ceb.2022.102081,2022-07-07,"[HBPsensing, PureMagRupture, MILIEU INTERIEUR,...",0955-0674,True
10,sdv,NaN,NaN,pasteur-03712988,UNDEFINED,10.1101/2021.08.30.458241,2022-07-04,[IIMory],None,False
15,sdv,American Society for Microbiology,2161-2129,pasteur-03675874,ART,10.1128/mbio.03510-21,2022-05-23,[IBEID],2150-7511,True
18,sdv,Public Library of Science,1935-2727,hal-03648068,ART,10.1371/journal.pntd.0010256,2022-04-21,[FutureHealthSEA],1935-2735,True
20,sdv.bbm.bc,Wiley-VCH Verlag,0265-9247,pasteur-03606359,ART,10.1002/bies.202100296,2022-03-11,[DEFineNMD],"0265-9247,1521-1878",True
23,phys,American Institute of Physics,0021-8979,hal-03532947,ART,10.1063/5.0063136,2022-02-21,[C-PI-GaN],"0021-8979,1089-7550",False
25,sdv.imm,Wiley-VCH Verlag,0014-2980,pasteur-03566563,ART,10.1002/eji.202149766,2022-02-11,"[PlanA, MILIEU INTERIEUR]","0014-2980,1521-4141",True
29,sdv.mp.pro,Nature Publishing Group,2058-5276,hal-03521407,ART,10.1038/s41564-021-01020-9,2022-01-13,"[METHEVOL, Arch-Evol]",2058-5276,True
35,sdv,NaN,NaN,hal-03472238,ART,10.1186/s13071-021-05042-4,2021-12-09,[FutureHealthSEA],1756-3305,True
46,sdv,American Society for Microbiology,0095-1137,pasteur-03346634,ART,10.1128/jcm.01581-21,2021-09-16,[IBEID],"0095-1137,1098-660X",True


# z-last step male table visible

In [23]:
df.columns

Index(['primaryDomain_s', 'journalPublisher_s', 'journalIssn_s', 'halId_s',
       'docType_s', 'doiId_s', 'submittedDate_s', 'anrProjectAcronym_s',
       'upw_issns', 'upw_vor_is_oa'],
      dtype='object')

In [46]:
export_df = df[["halId_s", "primaryDomain_s", "journalPublisher_s", "upw_vor_is_oa", "anrProjectAcronym_s", "submittedDate_s" ]]
export_df.rename(columns = 
                 {'halId_s':'halId', 
                  'primaryDomain_s':'primaryDomain', 
                  "journalPublisher_s" : "journalPublisher",
                  "upw_vor_is_oa" : "vorIsOa",
                  "anrProjectAcronym_s" : "anrProject",
                  "submittedDate_s" : "submittedDate"
                 }, 
                 inplace = True)

export_df["anrProject"].replace(to_replace= False, value="", inplace = True)
export_df.to_csv("../fr-publications-rrs-applied.csv", index = False)

c:\users\agentup\appdata\local\programs\python\python39\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
